In [2]:
from Helpers import pipelines
from Helpers import behave_metrics
from Helpers import shap_module
from Helpers import MetricsReport

In [2]:
nm = "LR"
print("-------------------- \n", f"{nm} is starting \n", "--------------------")

-------------------- 
 LR is starting 
 --------------------


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
import warnings
#from Helpers.radiomics_setup import RadiomicsClean, MultiParamProcessor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import os

import pickle
import yaml
# Define the hyperparameters for each model
hyperparameters_models_grid = {
    "Logistic Regression": {
        "classifier": LogisticRegression,
        "params": {
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'],
            'max_iter': [50, 100, 200, 300, 500],
            'l1_ratio': [0, 0.1, 0.2, 0.5, 0.7, 0.9, 1]
        },
        "default_params": {}
    },
    "Support Vector Machines": {
        "classifier": SVC,
        "params": {
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'degree': [2, 3, 4, 5],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100],
            'coef0': [0, 0.1, 0.5, 1, 2],
            'shrinking': [True, False],
            'tol': [1e-4, 1e-3, 1e-2, 1e-1],
            'probability': [True]
        },
        "default_params": {}
    },
    "Random Forest": {
        "classifier": RandomForestClassifier,
        "params": {
            'n_estimators': [20, 50, 100, 200, 300, 400, 500],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [None, 2, 4, 6, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10, 20],
            'min_samples_leaf': [1, 2, 4, 10],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
            'bootstrap': [True, False]
        },
        "default_params": {}
    },
    "Stochastic Gradient Descent": {
        "classifier": SGDClassifier,
        "params": {
            'loss': ['log', 'modified_huber'],
            'penalty': ['none', 'l2', 'l1', 'elasticnet'],
            'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
            'l1_ratio': [0.0, 0.15, 0.5, 0.85, 1.0],
            'fit_intercept': [True, False],
            'max_iter': [1000, 2000, 3000],
            'tol': [1e-3, 1e-4, 1e-5],
            'shuffle': [True, False],
            'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
            'eta0': [1e-4, 1e-3, 1e-2, 1e-1],
            'power_t': [0.25, 0.5, 0.75],
            'early_stopping': [True, False],
            'validation_fraction': [0.1, 0.2, 0.3],
            'n_iter_no_change': [5, 10, 20],
            'average': [False, True]
        },
        "default_params": {}
    },
    "Multi-Layer Neural Network": {
        "classifier": MLPClassifier,
        "params": {
            'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'solver': ['lbfgs', 'sgd', 'adam'],
            'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
            'learning_rate': ['constant', 'invscaling', 'adaptive'],
            'learning_rate_init': [0.001, 0.01, 0.1],
            'max_iter': [200, 300, 500, 1000],
            'shuffle': [True, False],
            'tol': [1e-4, 1e-3, 1e-2],
            'momentum': [0.9, 0.95, 0.99],
            'nesterovs_momentum': [True, False],
            'early_stopping': [True, False],
            'beta_1': [0.9, 0.95, 0.99],
            'beta_2': [0.999, 0.9995, 0.9999],
            'epsilon': [1e-8, 1e-7, 1e-6]
        },
        "default_params": {}
    },
    "Decision Trees": {
        "classifier": DecisionTreeClassifier,
        "params": {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'splitter': ['best', 'random'],
            'max_depth': [None, 2, 3, 5, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10, 20],
            'min_samples_leaf': [1, 2, 4, 10],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
            'max_leaf_nodes': [None, 2, 4, 10, 20, 30, 40, 50]
        },
        "default_params": {}
    },
    "XGBoost": {
        "classifier": XGBClassifier,
        "params": {
            'n_estimators': [3, 5, 10, 50, 100, 200, 300, 400, 500],
            'max_depth': [3, 4, 5, 6, 7, 8, 10],
            'learning_rate': [0.001, 0.01, 0.1, 0.3, 0.5],
            'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
            'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
            'reg_alpha': [0, 0.01, 0.1, 1, 10, 100],
            'reg_lambda': [0.01, 0.1, 1, 10, 100]
        },
        "default_params": {}
    }
}

yaml_file = os.path.join("./input_data", "machine_learning_parameters.yaml")
with open(yaml_file, 'r') as file:
    params = yaml.safe_load(file)

CORRELATION_LIMIT = params["Correlation Limit"]

NUMBER_OF_FOLDS = params["number_of_k_folds"]

if params["apply_grid_search"]["type"]["Randomized"]:
    HP_TYPE = "Randomized"
else:
    HP_TYPE = "Exhaustive"

names = []
classifiers = []
hypers = []

GRID_SEARCH_ENABLING = params["apply_grid_search"]["enabled"]

# Iterate over the machine learning models specified in the YAML file

for model_name, is_enabled in params["Machine Learning Models"].items():
    if is_enabled:
        model_info = hyperparameters_models_grid.get(model_name)
        if GRID_SEARCH_ENABLING:
            names.append(model_name)
            classifiers.append(model_info["classifier"])
            hypers.append(model_info["params"])
        else:
            names.append(model_name)
            classifiers.append(model_info["classifier"])
            hypers.append(model_info["default_params"])

In [12]:
names

['Support Vector Machines',
 'Stochastic Gradient Descent',
 'Multi-Layer Neural Network',
 'Decision Trees',
 'XGBoost']

In [ ]:
params["Machine Learning Models"]
for k,v in params["Machine Learning Models"].keys():
    print(item)

In [ ]:
params["number_of_k_folds"]
params["apply_grid_search"]["type"]["randomized"]

#params["Machine Learning Models"]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
import warnings
from Helpers.radiomics_setup import RadiomicsClean, MultiParamProcessor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import os
warnings.simplefilter(action='ignore', category=Warning)


In [ ]:
csv_paths = [
    r"Data\UC2_retrospective\ProCancer_UC2_N4_T2_DL_PZ_radio.csv",
    r"Data\UC2_retrospective\ProCancer_UC2_N4_ADC_DL_PZ_radio.csv",
    r"Data\UC2_retrospective\ProCancer_UC2_N4_DWI_DL_PZ_radio.csv",
]

exl = ["study_uid",
"number_of_series",
"provided_by",
"manufacturer",
"manufacturer_model_name",
"pi_rads"]

problematic = r"D:\dimza\EXPERIMENTS\ProstateCancerClinicalSignificance_UC2\UC2_prospective_and_retrospective\Borderline_cases_to_remove.csv"
jenn = r"Data\data_characteristics.csv"

mpp = MultiParamProcessor(
    csv_paths = csv_paths,
    problematic = problematic
)
mpp.iteration(
    suffices=["T2", "ADC", "DWI"],
    columns_to_exclude= exl,
    target_csv=jenn
)

dfs = mpp.get_dfs()

In [ ]:
#X_train, X_test, y_train, y_test
parent_folder = "./input_data"
train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)
train.to_csv(os.path.join(parent_folder,"Train.csv"))
test.to_csv(os.path.join(parent_folder,"Test.csv"))

In [ ]:
retro = pd.concat([dfs[0], dfs[1], dfs[2]], axis = 1)
X = retro.drop('Target', axis=1)  # Drop the 'Target' column for X_train
y = retro['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from Helpers import pipelines
from Helpers import behave_metrics
from Helpers import shap_module

In [ ]:
parent_folder = "./input_data"
train = pd.read_csv(os.path.join(parent_folder,"Train.csv"), index_col="patient_id")
test = pd.read_csv(os.path.join(parent_folder,"Test.csv"),index_col="patient_id")
X_train = train.drop('Target', axis=1)  # Drop the 'Target' column for X_train
y_train = train['Target']
X_test = test.drop('Target', axis=1)  # Drop the 'Target' column for X_train
y_test = test['Target']

In [ ]:
lr_hyperparameters = {'C': 1.0, 'penalty': 'l2'}
svm_hyperparameters = {'C': 1.0, 'kernel': 'rbf', 'probability':True}
xgb_hyperparameters = {'n_estimators': 100}
ada_hyperparameters = {'n_estimators': 50}
rf_hyperparameters = {'n_estimators': 100}
dt_hyperparameters = {'criterion': 'gini'}

names = ["Logistic Regression", "Support Vector Machines", "Random Forest", "AdaBoost", "Decision Trees", "XGBoost"]
classifiers = [LogisticRegression, SVC, RandomForestClassifier, AdaBoostClassifier, DecisionTreeClassifier, XGBClassifier]
hypers = [lr_hyperparameters , svm_hyperparameters, rf_hyperparameters, ada_hyperparameters, dt_hyperparameters, xgb_hyperparameters]

In [ ]:
lr_hyperparameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': [None, 'l2'],
    'solver': ['lbfgs','liblinear', 'saga','newton-cholesky'],
    'max_iter': [100, 200, 300],
    'class_weight': [None, 'balanced'],
}

svm_hyperparameters = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto'],
    'probability':[True],
}

rf_hyperparameters = {
    'n_estimators': [2,4,6,10,20,50],
    'max_depth': [None, 1,2,3, 4,6,8, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

ada_hyperparameters = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1],
    "n_jobs":[-1]
}

dt_hyperparameters = {
    'max_depth': [None, 2,3, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

xgb_hyperparameters = {
    'n_estimators': [3,5,10, 50],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [None, 1,2, 3, 6, 10, 20],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1],
}
hypers = [lr_hyperparameters , svm_hyperparameters, rf_hyperparameters, ada_hyperparameters, dt_hyperparameters, xgb_hyperparameters]

# CV-based on stratified K-Fold split and measurements on each fodl seperately

In [ ]:
pipeline_dict ={}
scores_storage = {}
params_dict = {}
thresholds = {}
# to add the automated k-fold selector based on the number of y
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state=10)
for cls, hp, nm in zip(classifiers, hypers, names):
    if nm=="Decision Trees":
    # find optimal parameters
        pipeline = pipelines.MLPipeline(X_train, y_train, cls, hp)
        pipeline.execute_feature_selection(corr_limit=0.6)
        pipeline.execute_preprocessing()
        pipeline.train_model(perform_grid_search=True, param_grid=hp, cv=skf)
        ppln = pipeline.build_pipeline()
        params = pipeline.get_best_parameters()
        params_dict.update({nm:params})

        hpers = params # best parameters based on cv grid search

        scores_storage_algo = {}
        thresholds_algo = {}
        for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
            xtrain = X_train.iloc[train_index,:]
            ytrain = y_train.iloc[train_index]
            xval = X_train.iloc[test_index,:]
            yval = y_train.iloc[test_index]

            pipeline = pipelines.MLPipeline(xtrain, ytrain, cls, hpers)
            pipeline.execute_feature_selection(corr_limit=0.6)
            pipeline.execute_preprocessing()
            pipeline.train_model()
            ppln = pipeline.build_pipeline()
            
            me = behave_metrics.ModelEvaluator(ppln,xval)
            scores = me.evaluate()["y_test"]

            # Find optimal threshold
            tho = behave_metrics.ThresholdOptimizer(ppln, xtrain, ytrain)
            thresh = tho.find_optimal_threshold(metric_to_track="AUC")
            thresholds_algo.update({f"fold_{i+1}":thresh})

            # Compute metrics on that threshold
            mr = behave_metrics.Metrics(scores, yval)
            mr.compute_metrics(threshold=thresh)
            scores_dict = mr.get_scores() # the scores on the fold based on the best hyperparameters
            scores_storage_algo.update({f"fold_{i+1}":scores_dict})
        scores_storage.update({nm:scores_storage_algo})
        thresholds.update({nm:thresholds_algo})

In [ ]:
from Helpers import MetricsReport
MetricsReport.summary_results_excel(scores_storage, file = "k_fold_results", conf_matrix_name="Internal_K_fold")

# Train on Whole X train and validation on the Holdout Set

In [ ]:
pipeline_dict_inf = {}
params_inf= {}
scores_inf = {}
for cls, hp, nm in zip(classifiers, hypers, names):
    if nm=="Decision Trees":
        hpers = params_dict[nm]
        pipeline = pipelines.MLPipeline(X_train, y_train, cls, hpers)
        pipeline.execute_feature_selection(corr_limit=0.6)
        pipeline.execute_preprocessing()
        pipeline.train_model(perform_grid_search=False)
        ppln = pipeline.build_pipeline()
        pipeline_dict_inf.update({nm:ppln})
        params_inf.update({nm:params})

        me = behave_metrics.ModelEvaluator(ppln,X_test)
        scores = me.evaluate()["y_test"]

        # Set optimal threshold as the average across Folds
        cnt = 0
        meas = 0
        for fold,val in thresholds[nm].items():
            cnt += 1
            meas += val
        average_threshold = meas/cnt if cnt!=0 else 0.5

        # Compute metrics on that threshold
        mr = behave_metrics.Metrics(scores, y_test)
        mr.compute_metrics(threshold=average_threshold)
        scores_dict = mr.get_scores() # the scores on the fold based on the best hyperparameters
        scores_inf.update({nm:scores_dict})

        # Perform Shap
        sp = shap_module.ShapAnalysis(X_val = X_test, pipeline_module = ppln, features=pipeline.selected_features) 
        shap_values,features = sp.perform_shap(), pipeline.selected_features

        os.mkdir(os.path.join("Materials", "Shap_Features"), exist_ok=True)
        path_shap = os.path.join("Materials", "Shap_Features")
        sp.plot_shap_values(model_name=nm, path= path_shap)

# display roc curves
roc = behave_metrics.ROCCurveEvaluator(pipeline_dict_inf,X_test=X_test, y_true=y_test)
roc.evaluate_models()
roc.plot_roc_curves(save_path=r"./Materials")



In [ ]:
MetricsReport.external_summary(scores_inf, file = "test_results", conf_matrix_name="Test")

# ____ End of Code ____

In [ ]:

path_to_csv = r"Data\UC2_retrospective\ProCancer_UC2_N4_ADC_DL_PZ_radio.csv"
problematic = r"D:\dimza\EXPERIMENTS\ProstateCancerClinicalSignificance_UC2\UC2_prospective_and_retrospective\Borderline_cases_to_remove.csv"
jenn = r"Data\data_characteristics.csv"

exl = ["study_uid",
"number_of_series",
"pi_rads"]

rdc = RadiomicsClean(path_to_csv)
rdc.clean_borderlines(problematic)
rdc.drop_columns_from_df(excluded_columns = exl)
#rdc.create_target()
rdc.create_target_based_on_csv(target_csv=jenn)
rdc.keep_zone_location(zone="PZ")
#rdc.add_suffix("T2")
features = rdc.get_processed()
kalliatakis = X_test.iloc[:,0:2]
kalliatakis["y_pred"] = scores
kalliatakis["y_true"] = y_test
kalliatakis.drop(columns=["gradient_firstorder_10Percentile_T2","gradient_firstorder_90Percentile_T2"],inplace=True)
df1 = features.loc[kalliatakis.index,["provided_by", "manufacturer", "manufacturer_model_name"]]
kalliatakis = pd.concat([df1,kalliatakis], axis=1)
kalliatakis = kalliatakis[kalliatakis.provided_by.isin(["QUIRONSALUD","UNIPI", "HULAFE", "FPO", "IDIBGI"])]
kalliatakis["Country"] = "Spain"
kalliatakis["Country"][kalliatakis.provided_by.isin(["UNIPI", "FPO"])] = "Italy"
kalliatakis.to_csv("ClinicalSignificant_ProstateCancer.csv")
kalliatakis[kalliatakis.provided_by.isin(["QUIRONSALUD","UNIPI", "HULAFE", "FPO", "IDIBGI"])]

In [ ]:
csv_paths = [
    r"Data\UC2_prospespective\ProCancer_UC2_N4_T2_DL_PZ_radio.csv",
    r"Data\UC2_prospespective\ProCancer_UC2_N4_ADC_DL_PZ_radio.csv",
    r"Data\UC2_prospespective\ProCancer_UC2_N4_DWI_DL_PZ_radio.csv",
]

mpp = MultiParamProcessor(
    csv_paths = csv_paths,
    problematic = problematic
)
mpp.iteration(
    suffices=["T2", "ADC", "DWI"],
    columns_to_exclude= exl
)

dfs = mpp.get_dfs()
prospe = pd.concat([dfs[0], dfs[1], dfs[2]], axis = 1)
X_test = prospe.drop('Target', axis=1)  # Drop the 'Target' column for X_train
y_test = prospe['Target']

evaluator = ModelEvaluator(pipeline, X_test)

# Evaluate the model on validation and test datasets
evaluation_results = evaluator.evaluate()

In [ ]:
evaluation_results